# Introduction to Convolutional Neural Networks

**Goal** : This notebook explains how to implement a simple convolutional neural network model. 

### History/Milestones for CNN

1. 1980 Kunihiko Fukushima introduction
2. 1998 Le Cun (Backpropagation) 
3. Contests won (2011& 2014 MINST Handwritten Dataset, Kaggle competitions, etc)
4. ImageNet success story : Alex Net(2012)

<img style="float: left;" src="img/1_cnn_accuracy_2012.png" height="520" width="520">

## MLP with Keras

In [2]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np
import pandas as pd

Using gpu device 0: GeForce GT 650M


### Load Data

In [3]:
from __future__ import absolute_import
import sys
import six.moves.cPickle
from six.moves import range
import os

def cifar10_load_batch(label_key='labels'):
    features_path = "../data/cifar_features"
    labels_path = "../data/cifar_labels"
    
    f = open(features_path, 'rb')
    if sys.version_info < (3,):
        d = six.moves.cPickle.load(f)
    else:
        d = six.moves.cPickle.load(f, encoding="bytes")
        # decode utf8
        for k, v in d.items():
            del(d[k])
            d[k.decode("utf8")] = v
    f.close()
    data = np.array(d)
    
    f = open(labels_path, 'rb')
    if sys.version_info < (3,):
        d = six.moves.cPickle.load(f)
    else:
        d = six.moves.cPickle.load(f, encoding="bytes")
        # decode utf8
        for k, v in d.items():
            del(d[k])
            d[k.decode("utf8")] = v
    f.close()
    labels = np.array(d)

    #data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels


def cifar10_load_data():

#     nb_test_samples = 10000
#     nb_train_samples = 50000

#     X_train = np.zeros((nb_train_samples, 3, 32, 32), dtype="uint8")
#     y_train = np.zeros((nb_train_samples,), dtype="uint8")

#     for i in range(1, 6):
#         fpath = os.path.join(path, 'data_batch_' + str(i))
#         data, labels = cifar10_load_batch(fpath)
#         X_train[(i-1)*10000:i*10000, :, :, :] = data
#         y_train[(i-1)*10000:i*10000] = labels
    
#     fpath = os.path.join(path, 'test_batch')
#     X_test, y_test = cifar10_load_batch(fpath)

#     y_train = np.reshape(y_train, (len(y_train), 1))
#     y_test = np.reshape(y_test, (len(y_test), 1))

    X_train, y_train = cifar10_load_batch()

    return (X_train, y_train)

In [4]:
(X_train, y_train) = cifar10_load_data()

In [5]:
print X_train.shape
print y_train.shape

(10000, 3, 32, 32)
(10000, 1)


In [6]:
X_train_reshaped = X_train.reshape(10000,3072)

In [7]:
X_train_reshaped.shape

(10000, 3072)

In [8]:
y_train.shape

(10000, 1)

### Data pre-processing

Create 2 categories of data:
* 5000 cats
* 5000 others

In [9]:
df = pd.DataFrame(X_train_reshaped, dtype=float)
df = pd.concat([df, pd.DataFrame(y_train)], axis=1)

df = df.reindex(np.random.permutation(df.index))
df.columns = range(0,3073)
cats = df[df[3072] == 3].copy()
cats[3072] = 1
dogs = df[df[3072] != 5]
dogs = dogs[:5000].copy()
dogs[3072] = 0
df = cats.append(dogs)
print df.shape

data = np.array(df.ix[:, :3071])
labels = np.array(df[3072])

data.shape
X_train = data
Y_train = np_utils.to_categorical(labels)
print X_train.shape
print Y_train.shape

(10000, 3073)
(10000, 3072)
(10000, 2)


In [10]:
# for faster convergence
dims = 64
# dims = 1500

Create a sequential model & add layers

In [11]:
model = Sequential()
model.add(Dense(X_train.shape[1], dims , init='uniform'))
model.add(Activation('relu'))

model.add(Dense(dims , dims, init='uniform'))
model.add(Activation('relu'))

model.add(Dense(dims, Y_train.shape[1], init='uniform'))
model.add(Activation('softmax'))

In [12]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

In [13]:
%timeit model.compile(loss='mean_squared_error', optimizer=sgd)

1 loops, best of 3: 2.65 s per loop


In [14]:
%timeit model.fit(X_train, Y_train, nb_epoch=2, show_accuracy=True)

Epoch 0
10000/10000 [==============================] - 1s - loss: 0.5006 - acc: 0.4993     
Epoch 1
10000/10000 [==============================] - 1s - loss: 0.5000 - acc: 0.5000     
Epoch 0
10000/10000 [==============================] - 1s - loss: 0.5000 - acc: 0.5000     
Epoch 1
10000/10000 [==============================] - 1s - loss: 0.5000 - acc: 0.5000     
Epoch 0
10000/10000 [==============================] - 1s - loss: 0.5000 - acc: 0.5000     
Epoch 1
10000/10000 [==============================] - 1s - loss: 0.5000 - acc: 0.5000     
Epoch 0
10000/10000 [==============================] - 1s - loss: 0.5000 - acc: 0.5000     
Epoch 1
10000/10000 [==============================] - 1s - loss: 0.5000 - acc: 0.5000     
1 loops, best of 3: 2.46 s per loop


# Convolutional Neural Network

In [15]:
import os
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.utils import np_utils

In [16]:
from keras.layers.convolutional import Convolution2D, MaxPooling2D

### fetch data

In [17]:
X_train, y_train = cifar10_load_data()

#### Each image has 3 channel (RGB) & is of size 32x32

In [18]:
X_train.shape

(10000, 3, 32, 32)

In [19]:
y_train[1]

array([3], dtype=uint32)

### Data pre-processing

Create 2 categories of data:
* 5000 cats
* 5000 others

In [20]:
z = zip(X_train, y_train)

In [21]:
def get_cats(x):
    if x[1][0] == 3:
        return True
    else:
        return False
    
def get_others(x):
    if x[1][0] != 3:
        return True
    else:
        return False
    

In [22]:
catz = filter(get_cats, z)
x_cat, y_cat = zip(*catz)

In [23]:
otherz = filter(get_others, z)
otherz = otherz[:5000]
x_other, y_other = zip(*otherz)

In [24]:
x_cats = np.array(x_cat)
x_others = np.array(x_other)

X_train = np.concatenate([x_cats,x_others])

In [25]:
X_train.shape

(10000, 3, 32, 32)

In [26]:
y_cats = np.array(y_cat)
y_others = np.array(y_other)

y_train = np.concatenate([y_cats,y_others])

In [27]:
y_train.shape

(10000, 1)

In [28]:
y_train[y_train == 3] = 1
y_train[y_train != 1] = 0

In [29]:
y_train

array([[1],
       [1],
       [1],
       ..., 
       [0],
       [0],
       [0]], dtype=uint32)

In [30]:
y_train.shape

(10000, 1)

#### Reshape y_train to represent 2 categories - Cats & Others

In [31]:
Y_train = np_utils.to_categorical(y_train, 2)
Y_train.shape

(10000, 2)

#### Create a sequential model, add convolutional layers with activation functions & regularizations

In [ ]:
#This works - Just one layer of Convolution
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 

model.add(Flatten())
model.add(Dense(32*34*34, 256))

model.add(Dense(256, 2))
model.add(Activation('softmax'))

X_train = X_train.astype("float32")
X_train /= 255

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(X_train, Y_train, batch_size=32, nb_epoch=1, show_accuracy=True)

In [55]:
#This works - Just one layer of Convolution and max pooling
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 
model.add(MaxPooling2D(poolsize=(2, 2)))

model.add(Flatten())
model.add(Dense(32*17*17, 256))

model.add(Dense(256, 2))
model.add(Activation('softmax'))

X_train = X_train.astype("float32")
X_train /= 255

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(X_train, Y_train, batch_size=32, nb_epoch=1, show_accuracy=True)

Epoch 0
10000/10000 [==============================] - 9s - loss: 6.7539 - acc: 0.5034     


In [57]:
#This works - Just one layer of Convolution but no full padding
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3)) 
#model.add(MaxPooling2D(poolsize=(2, 2)))

model.add(Flatten())
model.add(Dense(32*30*30, 256))

model.add(Dense(256, 2))
model.add(Activation('softmax'))

X_train = X_train.astype("float32")
X_train /= 255

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(X_train, Y_train, batch_size=32, nb_epoch=1, show_accuracy=True)

Epoch 0
10000/10000 [==============================] - 12s - loss: 7.9104 - acc: 0.5046    


In [54]:
#This works - all layers once
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 
#model.add(Convolution2D(32, 3, 3, 3)) 
#model.add(Activation('relu'))
#model.add(Convolution2D(32, 32, 3, 3))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(poolsize=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(32*34*34, 256))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))

model.add(Dense(256, 2))
model.add(Activation('softmax'))


X_train = X_train.astype("float32")
X_train /= 255

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(X_train, Y_train, batch_size=32, nb_epoch=1, show_accuracy=True)

Epoch 0
10000/10000 [==============================] - 16s - loss: 7.9239 - acc: 0.5000    


In [36]:
#This works - all layers once
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 
model.add(Activation('relu'))
model.add(Convolution2D(32, 32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(32*16*16, 256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256, 2))
model.add(Activation('softmax'))


X_train = X_train.astype("float32")
X_train /= 255

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(X_train, Y_train, batch_size=32, nb_epoch=1, show_accuracy=True)

Epoch 0
10000/10000 [==============================] - 18s - loss: 0.6971 - acc: 0.4949    


In [ ]:
# print X_train.shape, Y_train.shape

In [ ]:
#This is blank cell

In [ ]:

model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 
model.add(Activation('relu'))
model.add(Convolution2D(32, 32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 32, 3, 3, border_mode='full')) 
model.add(Activation('relu'))
model.add(Convolution2D(64, 64, 3, 3)) 
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64*8*8, 256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256, 2))
model.add(Activation('softmax'))






In [ ]:
X_train = X_train.astype("float32")
X_train /= 255

In [ ]:

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=1, show_accuracy=True)

### CNN with Dataset 2

In [ ]:
from __future__ import absolute_import
import os
import sys
import numpy as np
import scipy
import six.moves.cPickle
from six.moves import range

In [ ]:
with open("../data/rajni_cigar", "rb") as rf:
    l_data = six.moves.cPickle.loads(rf.read())

In [ ]:
l_data.shape

In [ ]:
unzipped = zip(*l_data)

In [ ]:
data = np.array(unzipped[0])

In [ ]:
labels = np.array(unzipped[1])

In [ ]:
data.shape

In [ ]:
labels.shape

In [ ]:
Y_LABELS = np_utils.to_categorical(labels, 2)

In [ ]:
Y_LABELS.shape

In [ ]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, 3, border_mode='full')) 
model.add(Activation('relu'))
model.add(Convolution2D(32, 32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 32, 3, 3, border_mode='full')) 
model.add(Activation('relu'))
model.add(Convolution2D(64, 64, 3, 3)) 
model.add(Activation('relu'))
model.add(MaxPooling2D(poolsize=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64*8*8, 256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256, 2))
model.add(Activation('softmax'))


In [ ]:
X_train = data
Y_train = Y_LABELS

In [ ]:
X_train = X_train.astype("float32")
X_train /= 255

In [ ]:

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=1, show_accuracy=True)

In [ ]:
df = pd.DataFrame([labels, predict]).T
df.columns = ['label', 'prediction']
df

### Data Augmentation

<img style="float: left;" src="img/2_data_augmentation.png" height="520" width="820">
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
[Source](http://benanne.github.io/2015/03/17/plankton.html)

# Momentum

Problem with Stochastic Gradient Descent are Valleys (local minima). Bounce up and down the walls and don‘t descent the slope. What's to our rescue? **Momentum**. Nesterov Momentum (NAG) 

<img style="float: left;" src="img/3_ momentum.png" height="520" width="620">